In [ ]:
import os
import sys

sys.path.append('..')

from src.utils.paths import get_project_path

from unstructured_ingest.v2.pipeline.pipeline import Pipeline
from unstructured_ingest.v2.interfaces import ProcessorConfig
from unstructured_ingest.v2.processes.connectors.local import (
    LocalIndexerConfig,
    LocalDownloaderConfig,
    LocalConnectionConfig,
    LocalUploaderConfig
)
from unstructured_ingest.v2.processes.partitioner import PartitionerConfig
from unstructured_ingest.v2.processes.chunker import ChunkerConfig

In [36]:
directory_with_pdfs= os.path.join(get_project_path(), 'data', 'Оригинальные статьи', 'ml_and_nanostructure')
directory_with_results = os.path.join(get_project_path(), 'data', 'parsed_pages')

# Pipeline.from_configs(
#     context=ProcessorConfig(),
#     indexer_config=LocalIndexerConfig(input_path=directory_with_pdfs),
#     downloader_config=LocalDownloaderConfig(),
#     source_connection_config=LocalConnectionConfig(),
#     partitioner_config=PartitionerConfig(
#         partition_by_api=True,
#         api_key=os.getenv("UNSTRUCTURED_API_KEY"),
#         partition_endpoint=os.getenv("UNSTRUCTURED_API_URL"),
#         strategy="hi_res",
#         additional_partition_args={
#             "split_pdf_page": True,
#             "split_pdf_concurrency_level": 15,
#             },
#         ),
#     uploader_config=LocalUploaderConfig(output_dir=directory_with_results)
# ).run()


In [37]:
from unstructured.staging.base import elements_from_json

def load_processed_files(directory_path):
    elements = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)
            try:
                elements.extend(elements_from_json(filename=file_path))
            except IOError:
                print(f"Error: Could not read file {filename}.")

    return elements

elements = load_processed_files(directory_with_results)

In [41]:
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

documents = []
for element in elements:
    metadata = element.metadata.to_dict()
    documents.append(Document(page_content=element.text, metadata=metadata))

db = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

INFO: Use pytorch device_name: cpu
INFO: Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
INFO: Loading faiss with AVX512 support.
INFO: Could not load library with AVX512 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx512'")
INFO: Loading faiss with AVX2 support.
INFO: Successfully loaded faiss with AVX2 support.


In [43]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
403 Client Error. (Request ID: Root=1-6725e8de-34b8924755e09bd9002c3506;db15ed41-98b3-40f7-b227-d2106bee3709)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Your request to access model meta-llama/Meta-Llama-3-8B-Instruct has been rejected by the repo's authors.

In [ ]:
for filename1 in os.listdir(os.path.join(get_project_path(), 'data', 'Оригинальные статьи')):
    print(filename1)
    for filename2 in os.listdir(os.path.join(get_project_path(), 'data', 'Оригинальные статьи', filename1)):
        if filename2.endswith('.pdf'):
            
        print(filename2)

2PP
2pp_for_biomedicine
materials_for_2pp
structure_created_by_2pp
methods_nanostructure
Conventional photolithography and reactive ion etching
High energy beam processing method
Hot embossing
idk
Interference lithography
Nanoimprinting
Thin film deposition processing technology
ml_and_nanostructure
2018_Generative Model for the Inverse Design of Metasurfaces.pdf
2018_Plasmonic nanostructure design and characterization via Deep Learning.pdf
2019_Deep learning for accelerated all-dielectric metasurface design.pdf
2019_Optimisation of colour generation from dielectric nanostructures using reinforcement learning.pdf
2019_Training artificial neural network for optimization of nanostructured VO2-based smart window performance.pdf
2020_Machine-Learning-Guided Morphology Engineering of Nanoscale Metal-Organic Frameworks.pdf
2020_Multitask deep-learning-based design of chiral plasmonic metamaterials.pdf
2020_Nanomaterial Synthesis Insights from Machine Learning of Scientific Articles by Extrac